In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import datetime
from sentinelhub import DataCollection, SHConfig, SentinelHubBYOC, BBox, CRS
import pandas

%matplotlib inline

from odc_sh import engine

Set Sentinel HUB credentials

In [ ]:
sh_client_id=""
sh_client_secret=""

if not sh_client_id:
    sh_client_id = os.environ['SH_CLIENT_ID']

if not sh_client_secret:
    sh_client_secret = os.environ['SH_CLIENT_SECRET']

Set id of your collection from the order

In [ ]:
collection_id = ""

In [ ]:
dc = engine.Datacube(sh_client_id=sh_client_id, sh_client_secret=sh_client_secret)

In [ ]:
myCollection = dc.get_BYOC_collection(collection_id)
myCollection

In [ ]:
#serch mapameters ... make 
resolution = 1  # in meters
longitude = (15.633142, 15.642218)
latitude = (46.406794, 46.413506)

time = ("2022-06-01", "2022-06-30")

ds = dc.load(
    product=myCollection,
    latitude=latitude,
    longitude=longitude,
    time=time,
    sh_resolution=resolution,
)

In [ ]:
# draw AIRBUS SPOT data
fig, ax = plt.subplots(nrows=1, ncols=len(ds.time), figsize=(12, 12), dpi=60)
for idx, t in enumerate(ds.time):
    single_time_frame = ds.sel(time=ds.time[idx], drop=True)
    
    weight = (single_time_frame.sel(bands=['B2']).data + single_time_frame.sel(bands=['B1']).data + single_time_frame.sel(bands=['B0']).data * 0.4) / 2.4;
    ratio = single_time_frame.sel(bands=['PAN']).data/weight * 2.5  ;
    
    res = single_time_frame.sel(bands=['B2', 'B1', 'B0']) / 10000 * ratio
    imgplot = res.plot.imshow(ax=ax, x="x", y="y", rgb="bands")
    imgplot.set_cmap('OrRd')
    plt.axis('off')
plt.show()